In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.metrics import R2Score
from sklearn.preprocessing import MinMaxScaler
#!pipx install optuna
#import optuna
from src import pre_process_ewt_data, prepare_data, read_yfinance_crypto_data, full_evaluation_flow, optimize_hyperparameters, split_data_set, model_builder

In [3]:
# Fetch Ripple price data
df = read_yfinance_crypto_data("ata-USD", "max", "1d")
scaler = MinMaxScaler()
vol_scaler = MinMaxScaler()

[
    [train_date_series, train_data, normalized_data],
    [test_date_series, testData, normalized_test_data]
] = pre_process_ewt_data(df, scaler, vol_scaler)

print(f"Train Data: {train_data.shape}\nTest Data: {testData.shape}")

test_target = testData
print(f"Train Target: {train_data.shape}\nTest Data: {test_target.shape}")

Cutoff:  393
Cutoff:  393
Cutoff:  393
Train Data: (919, 1)
Test Data: (393, 1)
Train Target: (919, 1)
Test Data: (393, 1)


In [ ]:
# Plot train data
plt.figure(figsize=(12, 6))
plt.plot(train_date_series, train_data[:], label='Closing')
plt.title('Crypto Daily Price Forecasting')
plt.xlabel('Date')
plt.ylabel('Price (USDT)')
plt.legend()

plt.figure(figsize=(12, 6))
plt.plot(train_date_series, normalized_data, label='NormalizedClosing')
plt.title('Normalized')
plt.xlabel('Date')
plt.ylabel('Norm. Y')
plt.legend()
plt.show()

In [4]:
# Prepare data for each EWT component
seq_len = 90
print(normalized_data.shape)
X, y, train_data, train_date_series = prepare_data(
    normalized_data,
    train_data,
    train_date_series,
    seq_len
)

print(y.shape, X.shape, train_date_series.shape, train_data.shape)

(919, 2)
(829, 1) (829, 90, 2) (829,) (829, 1)


In [5]:
# The training data is separated in a train set to train each opt iteration and a validation one
# to validate each iteration.

x_train, x_val = split_data_set(X, 0.3)
y_train, y_val = split_data_set(y, 0.3)

print(x_train.shape, x_val.shape)

Cutoff:  248
Cutoff:  248
(581, 90, 2) (248, 90, 2)


In [5]:

params = optimize_hyperparameters(
    model_builder,
    x_train,
    y_train,
    x_val,
    y_val,
    optuna
)

params

In [ ]:
full_evaluation_flow(
    model,
    prepare_data,
    normalized_test_data,
    testData,
    test_date_series,
    seq_len,
    "xrp-multi-feature-optimized-hyperp"
)

In [8]:
!mkdir -p saved_models
model.save("saved_models/xrp-multi-feat-opt-hyperp-p90-2L.keras")